<a href="https://colab.research.google.com/github/weltonrodrigo/first-order-model/blob/master/deepfake_babysteps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

💖 funciona em qualquer browser.
# <- Clica nessa pastinha aí (🗂) para abrir os arquivos 

<h2>Arrasta pra dentro dois arquivos, um chamado video.mp4 e o outro chamado foto.jpg</h2>
Não importa o formato do arquivo, só coloca eles com o nome video.mp4 e foto.jpg

Para gerar seu deepfake, você só precisa:

1.   Ir no menu **Runtime -> Run all**
2.   Aguardar o resultado ficar pronto, no [<em>🥌 final da página</em>](#scrollTo=6pwxZtBCGBJB)

Este caderno é baseado no original [Demo for paper "First Order Motion Model for Image Animation"](https://colab.research.google.com/github/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb#scrollTo=cdO_RxQZLahB) 

### Dicas para um bom resultado:

- O corte automático às vezes não funciona bem, se você puder, use uma foto já
cortada, 256 por 256 pixels, do pescoço pra cima, sem mostrar muito os ombros. Foi com esse tipo de foto que o modelo foi treinado, então o desempenho é melhor com fotos parecidas.
- O ideial é o video tb ter essas características e começar com uma expressão parecida com a da foto. Você pode imitar a pessoa da foto e depois se movimentar para falar ou fazer o que quiser, depois editar o resultado.
- Vídeo com o rosto claro, bem iluminado.
- Fotos onde há uma boa diferença entre a pessoa e o fundo.
- Fotos onde a pessoa está numa expressão neutra.

Exemplo de foto cortada adequadamente:

<img src="https://i.stack.imgur.com/ds7Pc.png" alt="Foto bem fechada no rosto" width="256"/>


Tutorial: https://youtu.be/o6o0aLHt734


---

### Download da rede neural pré-treinada. 
Isso deve levar uns 4 minutos quando rodar a primeira vez.

In [0]:
!curl -C - -L https://1drv.ws/u/s!AsP1keFmL5J8gZkisFuiF-Yecx8YIQ?e=xjKj3u  -o /content/vox-adv-cpk.pth.tar

# Um monte de coisa preparada antes que você não precisa mexer.

### Variáveis interessantes

In [0]:
# Quando um rosto é encontrado na foto ou no vídeo, a área recortada é aumentada
# nessa porcentagem. Aumente para diminuir o zoom, Diminua para um zoom menor
INCREASE = 0.1

### Processamento

In [0]:
!pip install face_alignment imageio-ffmpeg

In [0]:
%cd /content
!rm -rf /content/first-order-model
!mkdir -p /content/first-order-model
!git clone https://github.com/AliaksandrSiarohin/first-order-model /content/first-order-model
%cd /content/first-order-model

In [0]:
from argparse import ArgumentParser
from base64 import b64encode
from demo import load_checkpoints, make_animation
from IPython.display import HTML
from multiprocessing import Process
from PIL import Image 
from skimage import img_as_ubyte, io
from skimage.transform import resize
from tqdm import tqdm
import face_alignment
import glob
import imageio
import matplotlib.animation as animation
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import subprocess
import sys
import warnings
warnings.filterwarnings("ignore")

In [0]:
generator, kp_detector = load_checkpoints(config_path='config/vox-adv-256.yaml', checkpoint_path='/content/vox-adv-cpk.pth.tar')

In [0]:
SOURCE_VIDEO_FILENAME = "/content/video.mp4"

In [0]:
SOURCE_IMAGE_FILENAME = "/content/foto.jpg"

# Recortando o rosto no vídeo.
Isso deve demorar um pouco

In [0]:
%%capture --no-display FFMPEG_COMMAND
!python crop-video.py --inp $SOURCE_VIDEO_FILENAME --increase=$INCREASE | grep ffmpeg

# Continuando o processamento

In [0]:
print(FFMPEG_COMMAND.stderr)

In [0]:
print(FFMPEG_COMMAND.stdout)

In [0]:
import re
p = re.compile('(ffmpeg.*)')
m = p.search(FFMPEG_COMMAND.stdout)
command = m.group(1)
LESS_VERBOSE = command.replace('ffmpeg', 'ffmpeg -hide_banner -loglevel warning -y ')

In [0]:
LESS_VERBOSE

In [0]:
# LESS_VERBOSE = FFMPEG_COMMAND.stdout.replace('ffmpeg', 'ffmpeg -hide_banner -loglevel warning -y ')
!$LESS_VERBOSE

In [0]:
def compute_bbox(tube_bbox, frame_shape, increase_area=0.2):
    left, top, right, bot, _ = tube_bbox
    width = right - left
    height = bot - top 

    #Computing aspect preserving bbox 
    width_increase = max(increase_area, ((1 + 2 * increase_area) * height - width) / (2 * width))
    height_increase = max(increase_area, ((1 + 2 * increase_area) * width - height) / (2 * height))

    left = int(left - width_increase * width)
    top = int(top - height_increase * height)
    right = int(right + width_increase * width)
    bot = int(bot + height_increase * height)

    top, bot, left, right = max(0, top), min(bot, frame_shape[0]), max(0, left), min(right, frame_shape[1])
    h, w = bot - top, right - left

    # print (f'-filter:v "crop={w}:{h}:{left}:{top}"')
    return (left, top, right, bot)

## Se mudou só a imagem, pode rodar denovo só a partir daqui.

In [0]:
image = io.imread(SOURCE_IMAGE_FILENAME)

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, device='cuda')
bboxes = fa.face_detector.detect_from_image(image[..., :3])

In [0]:
# image = io.imread(SOURCE_IMAGE_FILENAME)

# fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, device='cuda')
# bboxes = fa.face_detector.detect_from_image(image[..., :3])

(left, top, right, bot) = compute_bbox(bboxes[0], image.shape[0:2], increase_area=INCREASE)

source_image = image[top:bot,left:right,:]

io.imsave("cropped_image.png", source_image)
plt.imshow(source_image)

In [0]:
(left, top, right, bot)

In [0]:
reader = imageio.get_reader('crop.mp4')
FPS = reader.get_meta_data()['fps']
reader.close()

driving_video = imageio.mimread('crop.mp4')


source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

# Gerando o deepfake


In [0]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
imageio.mimsave('/content/generated-relative.mp4', [img_as_ubyte(frame) for frame in predictions], fps=FPS)

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
imageio.mimsave('/content/generated-absolute.mp4', [img_as_ubyte(frame) for frame in predictions], fps=FPS)

# Gerando o vídeo lado a lado

In [0]:
%%bash
ffmpeg \
-hide_banner \
-y \
-loglevel error \
-i cropped_image.png \
-hwaccel cuvid \
-i crop.mp4 \
-i /content/generated-relative.mp4 \
-i https://i.ibb.co/x2ndW9G/disclaimer.png \
-async 1 \
-filter_complex  "
nullsrc=size=768x256                  [bg];
[0]scale=256:256                      [srcimg];
[3]format=bgra,colorchannelmixer=aa=0.5 [disclaimer];
[bg][srcimg]       overlay            [bg+srcimg];
[bg+srcimg][1:v]   overlay=x=256:shortest=1      [left+middle];
[left+middle][2]   overlay=x=512      [left+middle+right];
[left+middle+right][disclaimer]overlay=x=512 [resultado]  
" \
-map '[resultado]' -map 1:a:0?  \
-c:v h264 -preset veryslow  -c:a copy -metadata title="https://bit.ly/2Vars0s" -y /content/complete-relative.mp4;



In [0]:
%%bash
ffmpeg \
-hide_banner \
-y \
-loglevel error \
-i cropped_image.png \
-hwaccel cuvid \
-i crop.mp4 \
-i /content/generated-absolute.mp4 \
-i https://i.ibb.co/x2ndW9G/disclaimer.png \
-async 1 \
-filter_complex  "
nullsrc=size=768x256                  [bg];
[0]scale=256:256                      [srcimg];
[3]format=bgra,colorchannelmixer=aa=0.5 [disclaimer];
[bg][srcimg]       overlay            [bg+srcimg];
[bg+srcimg][1:v]   overlay=x=256:shortest=1      [left+middle];
[left+middle][2]   overlay=x=512      [left+middle+right];
[left+middle+right][disclaimer]overlay=x=512 [resultado]   
" \
-map '[resultado]' -map 1:a:0?  \
-c:v h264 -preset veryslow  -c:a copy -y -metadata title="https://bit.ly/2Vars0s" /content/complete-absolute.mp4;

In [0]:
relative = open('/content/complete-relative.mp4','rb').read()
data_url_relative = "data:video/mp4;base64," + b64encode(relative).decode()

absolute = open('/content/complete-absolute.mp4','rb').read()
data_url_absolute = "data:video/mp4;base64," + b64encode(absolute).decode()

# RESULTADO
Deve demorar uns 5 minutos

In [0]:
HTML("""
<video controls="controls" autoplay="autoplay">
      <source src="%s" type="video/mp4">
</video>
<br /><h1><a target="_blank" href="%s" download="deep_fake_relativo.mp4">➡️Clique aqui baixar para o resultado relativo.⬅️</a><h1>
<video controls="controls" autoplay="autoplay">
      <source src="%s" type="video/mp4">
</video>
<br /><h1><a target="_blank" href="%s" download="deep_fake_absoluto.mp4">➡️Clique aqui baixar para o resultado absoluto.⬅️</a><h1>
""" % (data_url_relative, data_url_relative, data_url_absolute, data_url_absolute))